In [1]:
from model.model_builder import DeeplabV3PlusModel

model = DeeplabV3PlusModel()

/home/hyunho/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \
/home/hyunho/sfda/model/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/hyunho/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
/home/hyunho/sfda/model/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 

In [2]:
print(model.decode_head)

DepthwiseSeparableASPPHead(
  input_transform=None, ignore_index=255, align_corners=False
  (loss_decode): CrossEntropyLoss(avg_non_ignore=False)
  (conv_seg): Conv2d(512, 19, kernel_size=(1, 1), stride=(1, 1))
  (dropout): Dropout2d(p=0.1, inplace=False)
  (image_pool): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): ConvModule(
      (conv): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): SyncBatchNorm(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
  )
  (aspp_modules): DepthwiseSeparableASPPModule(
    (0): ConvModule(
      (conv): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): SyncBatchNorm(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (1): DepthwiseSeparableConvModule(
      (depthwise_conv): ConvModule(
        (conv): Conv2d(2048, 2048, kernel_size=(3, 3), stride=(1, 1), p

In [3]:
import torch
checkpoint = torch.load("/home/hyunho/sfda/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth", weights_only=True)

In [4]:
model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [5]:
model.eval()

DeeplabV3PlusModel(
  (backbone): ResNetV1c(
    (stem): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): SyncBatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): SyncBatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (7): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, tra

In [45]:
from PIL import Image
import torchvision.transforms as transforms
import torch

# 이미지 파일 열기
img_path = "/home/hyunho/sfda/data/gta5_dataset/images/train/00001.png"  # 이미지를 테스트할 경로
img = Image.open(img_path)

# 이미지 크기를 512x1024로 변경하고 텐서로 변환
transform = transforms.Compose([
    transforms.Resize((512, 1024)),  # 크기 조정
    transforms.ToTensor()            # 텐서로 변환
])

img_tensor = transform(img)
img_tensor = img_tensor.unsqueeze(0)

In [46]:
img_tensor.shape

torch.Size([1, 3, 512, 1024])

In [55]:
output = model(img_tensor)

In [56]:
len(output)

1

In [57]:
output[0].shape

torch.Size([19, 128, 256])

In [59]:
model.auxiliary_head._forward_feature(output)

RuntimeError: Given groups=1, weight of size [256, 1024, 3, 3], expected input[1, 19, 128, 256] to have 1024 channels, but got 19 channels instead